# Install Ultralytics

In [23]:
pip install ultralytics

# Move Files

In [24]:
import os
import zipfile
import shutil

def unzip_and_move(zip_path, dest_dir="data"):
    # Create the destination directory if it doesn't exist
    os.makedirs(dest_dir, exist_ok=True)

    # Extract the zip file
    with zipfile.ZipFile(zip_path, 'r') as zip_ref:
        zip_ref.extractall(dest_dir)

    print(f"Extracted contents of {zip_path} to {dest_dir}")

if __name__ == "__main__":
    zip_file = "upload.zip"  # Change this to your zip file path
    unzip_and_move(zip_file)


Extracted contents of upload.zip to data


# Update Labels & Create yaml

In [36]:
import os

# Generate data.yaml
#YOLO Training yaml 格式官網
#https://docs.ultralytics.com/yolov5/tutorials/train_custom_data/#3-select-a-model

def get_class_mapping(data_dir):
    """ Get a mapping of folder names to new class indices. """
    class_folders = sorted([folder for folder in os.listdir(data_dir) if os.path.isdir(os.path.join(data_dir, folder))])
    return {folder: idx for idx, folder in enumerate(class_folders)}

def update_labels(data_dir, class_mapping):
    """ Update label files by replacing the first value (class index) in each line. """
    for class_name, new_idx in class_mapping.items():
        label_folder = os.path.join(data_dir, class_name, "labels")

        if not os.path.exists(label_folder):
            continue  # Skip if no "labels" folder

        for label_file in os.listdir(label_folder):
            label_path = os.path.join(label_folder, label_file)

            if label_file.endswith(".txt"):
                with open(label_path, "r") as f:
                    lines = f.readlines()

                updated_lines = []
                for line in lines:
                    parts = line.strip().split()  # Split by space
                    if parts:  # Ensure it's not an empty line
                        parts[0] = str(new_idx)  # Replace class index
                        updated_lines.append(" ".join(parts) + "\n")

                with open(label_path, "w") as f:
                    f.writelines(updated_lines)

                print(f"Updated {label_path} -> New class index: {new_idx}")

def create_yolo_yaml(data_dir, class_mapping):
    """ Create a data.yaml file for YOLO training. """
    yaml_path = os.path.join(data_dir, "data.yaml")

    yaml_content = f"""
path: /content
train: train_data
val: val_data
test: test_data

nc: {len(class_mapping)}  # Number of classes
names: {list(class_mapping.keys())}  # Class names
"""

    with open(yaml_path, "w") as f:
        f.write(yaml_content.strip())

    print(f"Created YOLO data.yaml at {yaml_path}")

if __name__ == "__main__":
    data_directory = "data"
    class_mapping = get_class_mapping(data_directory)
    print("Class Mapping:", class_mapping)

    update_labels(data_directory, class_mapping)
    print("Label updating complete.")

    create_yolo_yaml(data_directory, class_mapping)
    print("YOLO data.yaml file created successfully.")


# Move Class Images into Single Folder

In [26]:
import os
import shutil

def move_files(source_root, dest_folder, file_ext):
    """Moves all files with a given extension from subfolders into a destination folder."""
    os.makedirs(dest_folder, exist_ok=True)

    for class_folder in os.listdir(source_root):
        class_path = os.path.join(source_root, class_folder)

        if not os.path.isdir(class_path):
            continue  # Skip non-directory files

        for subfolder in ["images", "labels"]:  # Look for images and labels subfolders
            subfolder_path = os.path.join(class_path, subfolder)
            if not os.path.exists(subfolder_path):
                continue  # Skip if subfolder doesn't exist

            for file in os.listdir(subfolder_path):
                if file.endswith(file_ext):
                    src_file = os.path.join(subfolder_path, file)
                    dest_file = os.path.join(dest_folder, file)

                    # Handle duplicate filenames by renaming
                    if os.path.exists(dest_file):
                        base, ext = os.path.splitext(file)
                        counter = 1
                        while os.path.exists(dest_file):
                            dest_file = os.path.join(dest_folder, f"{base}_{counter}{ext}")
                            counter += 1

                    shutil.move(src_file, dest_file)
                    print(f"Moved {src_file} → {dest_file}")

if __name__ == "__main__":
    dataset_root = "data"
    images_dest = os.path.join(dataset_root, "images")
    labels_dest = os.path.join(dataset_root, "labels")

    move_files(dataset_root, images_dest, ".jpg")  # Move images
    move_files(dataset_root, images_dest, ".png")  # Support PNGs too
    move_files(dataset_root, labels_dest, ".txt")  # Move labels

    print("All files have been moved successfully!")


Moved data/fish/images/6b12cc00-fish.233.jpg → data/images/6b12cc00-fish.233.jpg
Moved data/fish/images/5b6fff43-fish.14.jpg → data/images/5b6fff43-fish.14.jpg
Moved data/fish/images/f7a20f4c-fish.278.jpg → data/images/f7a20f4c-fish.278.jpg
Moved data/fish/images/14d9fe45-fish.146.jpg → data/images/14d9fe45-fish.146.jpg
Moved data/fish/images/402d30c9-fish.387.jpg → data/images/402d30c9-fish.387.jpg
Moved data/fish/images/3c83f851-fish.152.jpg → data/images/3c83f851-fish.152.jpg
Moved data/fish/images/6ffec556-fish.336.jpg → data/images/6ffec556-fish.336.jpg
Moved data/fish/images/c9ac78e1-fish.37.jpg → data/images/c9ac78e1-fish.37.jpg
Moved data/fish/images/18af6ea0-fish.380.jpg → data/images/18af6ea0-fish.380.jpg
Moved data/fish/images/c9a1fa40-fish.27.jpg → data/images/c9a1fa40-fish.27.jpg
Moved data/fish/images/669c72b6-fish.350.jpg → data/images/669c72b6-fish.350.jpg
Moved data/fish/images/0f073523-fish.28.jpg → data/images/0f073523-fish.28.jpg
Moved data/fish/images/1cf895a2-fish

# Data Augmentation

In [28]:
import os
import cv2

def flip_yolo_labels(label_file, flip_type):
    """ Adjust YOLO labels based on the type of flip applied. """
    with open(label_file, "r") as f:
        lines = f.readlines()

    flipped_lines = []
    for line in lines:
        parts = line.strip().split()
        if len(parts) == 5:
            class_id = parts[0]
            x_center = float(parts[1])
            y_center = float(parts[2])
            width = float(parts[3])
            height = float(parts[4])

            if flip_type == "h":  # Horizontal flip
                x_center = 1 - x_center
            elif flip_type == "v":  # Vertical flip
                y_center = 1 - y_center
            elif flip_type == "hv":  # Both flips
                x_center = 1 - x_center
                y_center = 1 - y_center

            flipped_lines.append(f"{class_id} {x_center:.6f} {y_center:.6f} {width:.6f} {height:.6f}\n")

    return flipped_lines

def apply_augmentations(image_folder, label_folder, output_image_folder, output_label_folder):
    """ Apply flips and save augmented images and updated labels. """
    os.makedirs(output_image_folder, exist_ok=True)
    os.makedirs(output_label_folder, exist_ok=True)

    for image_file in os.listdir(image_folder):
        if image_file.endswith((".jpg", ".png")):
            image_path = os.path.join(image_folder, image_file)
            label_path = os.path.join(label_folder, image_file.replace(".jpg", ".txt").replace(".png", ".txt"))

            if not os.path.exists(label_path):
                continue  # Skip if label file doesn't exist

            image = cv2.imread(image_path)

            # Save original image and label
            base_name = os.path.splitext(image_file)[0]
            cv2.imwrite(os.path.join(output_image_folder, image_file), image)
            with open(os.path.join(output_label_folder, base_name + ".txt"), "w") as f:
                f.writelines(open(label_path, "r").readlines())

            # Horizontal flip
            h_image = cv2.flip(image, 1)
            h_label = flip_yolo_labels(label_path, "h")
            cv2.imwrite(os.path.join(output_image_folder, base_name + "_h.jpg"), h_image)
            with open(os.path.join(output_label_folder, base_name + "_h.txt"), "w") as f:
                f.writelines(h_label)

            # Vertical flip
            v_image = cv2.flip(image, 0)
            v_label = flip_yolo_labels(label_path, "v")
            cv2.imwrite(os.path.join(output_image_folder, base_name + "_v.jpg"), v_image)
            with open(os.path.join(output_label_folder, base_name + "_v.txt"), "w") as f:
                f.writelines(v_label)

            # Both flips (Horizontal + Vertical)
            hv_image = cv2.flip(image, -1)
            hv_label = flip_yolo_labels(label_path, "hv")
            cv2.imwrite(os.path.join(output_image_folder, base_name + "_hv.jpg"), hv_image)
            with open(os.path.join(output_label_folder, base_name + "_hv.txt"), "w") as f:
                f.writelines(hv_label)

            print(f"Augmented: {image_file}")

if __name__ == "__main__":
    image_folder = "data/images"
    label_folder = "data/labels"
    output_image_folder = "data/augmented_images"
    output_label_folder = "data/augmented_labels"

    apply_augmentations(image_folder, label_folder, output_image_folder, output_label_folder)

    print("Data augmentation complete.")


Augmented: c736b2d0-eggs.163.jpg
Augmented: d6bd16e1-eggs.91.jpg
Augmented: d7a04d7f-eggs.174.jpg
Augmented: 6b12cc00-fish.233.jpg
Augmented: e14a498c-eggs.200.jpg
Augmented: 6730c299-eggs.151.jpg
Augmented: 5b6fff43-fish.14.jpg
Augmented: f7a20f4c-fish.278.jpg
Augmented: f3352a94-eggs.117.jpg
Augmented: 14d9fe45-fish.146.jpg
Augmented: 74f20445-eggs.30.jpg
Augmented: 402d30c9-fish.387.jpg
Augmented: 3c83f851-fish.152.jpg
Augmented: 22f8924d-eggs.79.jpg
Augmented: 6ffec556-fish.336.jpg
Augmented: c9ac78e1-fish.37.jpg
Augmented: e1d3661e-eggs.120.jpg
Augmented: 18af6ea0-fish.380.jpg
Augmented: c9a1fa40-fish.27.jpg
Augmented: b6d5e067-eggs.42.jpg
Augmented: 4c209381-eggs.136.jpg
Augmented: 9cea7c4d-eggs.83.jpg
Augmented: 828961e1-eggs.77.jpg
Augmented: 071e865b-eggs.167.jpg
Augmented: 4e80c34c-eggs.3.jpg
Augmented: d9cee69d-eggs.106.jpg
Augmented: 669c72b6-fish.350.jpg
Augmented: e3a44ccd-eggs.16.jpg
Augmented: 6bbe1162-eggs.149.jpg
Augmented: 3419f0e1-eggs.175.jpg
Augmented: 0f073523-fi

# Move Augmented Images/Labels

In [29]:
import os
import shutil

# Define source and destination directories
image_src = "data/augmented_images"
label_src = "data/augmented_labels"
image_dest = "data/images"
label_dest = "data/labels"

# Ensure destination directories exist
os.makedirs(image_dest, exist_ok=True)
os.makedirs(label_dest, exist_ok=True)

# Move images
for file in os.listdir(image_src):
    src_path = os.path.join(image_src, file)
    dest_path = os.path.join(image_dest, file)

    if os.path.isfile(src_path):
        shutil.move(src_path, dest_path)
        print(f"Moved: {src_path} -> {dest_path}")

# Move labels
for file in os.listdir(label_src):
    src_path = os.path.join(label_src, file)
    dest_path = os.path.join(label_dest, file)

    if os.path.isfile(src_path):
        shutil.move(src_path, dest_path)
        print(f"Moved: {src_path} -> {dest_path}")

print("All files moved successfully!")

Moved: data/augmented_images/dd06ef22-fish.39_v.jpg -> data/images/dd06ef22-fish.39_v.jpg
Moved: data/augmented_images/702da948-fish.332_v.jpg -> data/images/702da948-fish.332_v.jpg
Moved: data/augmented_images/c736b2d0-eggs.163.jpg -> data/images/c736b2d0-eggs.163.jpg
Moved: data/augmented_images/2106e0b2-fish.369_v.jpg -> data/images/2106e0b2-fish.369_v.jpg
Moved: data/augmented_images/c7427bec-eggs.61_v.jpg -> data/images/c7427bec-eggs.61_v.jpg
Moved: data/augmented_images/4df044c4-eggs.135_h.jpg -> data/images/4df044c4-eggs.135_h.jpg
Moved: data/augmented_images/d6cfdec8-fish.59_hv.jpg -> data/images/d6cfdec8-fish.59_hv.jpg
Moved: data/augmented_images/d6bd16e1-eggs.91.jpg -> data/images/d6bd16e1-eggs.91.jpg
Moved: data/augmented_images/d7a04d7f-eggs.174.jpg -> data/images/d7a04d7f-eggs.174.jpg
Moved: data/augmented_images/6b12cc00-fish.233.jpg -> data/images/6b12cc00-fish.233.jpg
Moved: data/augmented_images/d04f2519-eggs.176_h.jpg -> data/images/d04f2519-eggs.176_h.jpg
Moved: dat

# Data Split

In [30]:
import os
import shutil
import random

# Define dataset directory
dataset_dir = "data"
images_dir = os.path.join(dataset_dir, "images")
labels_dir = os.path.join(dataset_dir, "labels")

# Define split percentages
train_ratio = 0.7
val_ratio = 0.15
test_ratio = 0.15

# Output directories
output_dirs = {
    "train": os.path.join(dataset_dir, "../train_data"),
    "val": os.path.join(dataset_dir, "../val_data"),
    "test": os.path.join(dataset_dir, "../test_data"),
}

# Create subdirectories
for split in output_dirs.values():
    os.makedirs(os.path.join(split, "images"), exist_ok=True)
    os.makedirs(os.path.join(split, "labels"), exist_ok=True)

# Get all image filenames (assumes .jpg, change if needed)
image_filenames = [f for f in os.listdir(images_dir) if f.endswith((".jpg", ".png"))]

# Shuffle data to ensure randomness
random.shuffle(image_filenames)

# Split data
total_files = len(image_filenames)
train_count = int(total_files * train_ratio)
val_count = int(total_files * val_ratio)

train_files = image_filenames[:train_count]
val_files = image_filenames[train_count:train_count + val_count]
test_files = image_filenames[train_count + val_count:]

# Function to move files
def move_files(file_list, split_name):
    for file in file_list:
        img_src = os.path.join(images_dir, file)
        label_src = os.path.join(labels_dir, file.replace(".jpg", ".txt").replace(".png", ".txt"))

        img_dest = os.path.join(output_dirs[split_name], "images", file)
        label_dest = os.path.join(output_dirs[split_name], "labels", os.path.basename(label_src))

        shutil.move(img_src, img_dest)
        if os.path.exists(label_src):  # Ensure label exists before moving
            shutil.move(label_src, label_dest)

# Move files into their respective directories
move_files(train_files, "train")
move_files(val_files, "val")
move_files(test_files, "test")

print("Dataset split complete!")
print(f"Train: {len(train_files)}, Val: {len(val_files)}, Test: {len(test_files)}")


Dataset split complete!
Train: 887, Val: 190, Test: 191


# Train, Eval, and Save

In [ ]:
from ultralytics import YOLO
# Step 3: Load YOLOv11 Model
model = YOLO("yolo11n.pt")  # Use YOLOv11 nano for training

# Step 4: Train Model
model.train(data="data.yaml", epochs=50, imgsz=640, batch=64)

#didn't delete noise

# Step 5: Evaluate Model
metrics = model.val()
print("Evaluation Metrics:", metrics)

# Step 6: Save Model
model.export(format='onnx')  # Save as ONNX for deployment

Ultralytics 8.3.92 🚀 Python-3.11.11 torch-2.6.0+cu124 CUDA:0 (Tesla T4, 15095MiB)
engine/trainer: task=detect, mode=train, model=yolo11n.pt, data=data.yaml, epochs=50, time=None, patience=100, batch=64, imgsz=640, save=True, save_period=-1, cache=False, device=None, workers=8, project=None, name=train4, exist_ok=False, pretrained=True, optimizer=auto, verbose=True, seed=0, deterministic=True, single_cls=False, rect=False, cos_lr=False, close_mosaic=10, resume=False, amp=True, fraction=1.0, profile=False, freeze=None, multi_scale=False, overlap_mask=True, mask_ratio=4, dropout=0.0, val=True, split=val, save_json=False, save_hybrid=False, conf=None, iou=0.7, max_det=300, half=False, dnn=False, plots=True, source=None, vid_stride=1, stream_buffer=False, visualize=False, augment=False, agnostic_nms=False, classes=None, retina_masks=False, embed=None, show=False, save_frames=False, save_txt=False, save_conf=False, save_crop=False, show_labels=True, show_conf=True, show_boxes=True, line_widt

100%|██████████| 755k/755k [00:00<00:00, 21.5MB/s]


Overriding model.yaml nc=80 with nc=6

                   from  n    params  module                                       arguments                     
  0                  -1  1       464  ultralytics.nn.modules.conv.Conv             [3, 16, 3, 2]                 
  1                  -1  1      4672  ultralytics.nn.modules.conv.Conv             [16, 32, 3, 2]                
  2                  -1  1      6640  ultralytics.nn.modules.block.C3k2            [32, 64, 1, False, 0.25]      
  3                  -1  1     36992  ultralytics.nn.modules.conv.Conv             [64, 64, 3, 2]                
  4                  -1  1     26080  ultralytics.nn.modules.block.C3k2            [64, 128, 1, False, 0.25]     
  5                  -1  1    147712  ultralytics.nn.modules.conv.Conv             [128, 128, 3, 2]              
  6                  -1  1     87040  ultralytics.nn.modules.block.C3k2            [128, 128, 1, True]           
  7                  -1  1    295424  ultralytics

train: Scanning /content/train_data/labels... 887 images, 0 backgrounds, 2 corrupt: 100%|██████████| 887/887 [00:00<00:00, 2624.70it/s]

train: WARNING ⚠️ /content/train_data/images/93867eb8-fish.388.jpg: ignoring corrupt image/label: non-normalized or out of bounds coordinates [     1.0762]
train: WARNING ⚠️ /content/train_data/images/93867eb8-fish.388_h.jpg: ignoring corrupt image/label: negative label values [  -0.076239]
train: New cache created: /content/train_data/labels.cache


...


      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      28/50      10.8G     0.9129     0.6751      1.168        271        640: 100%|██████████| 14/14 [00:11<00:00,  1.20it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 2/2 [00:02<00:00,  1.29s/it]

                   all        189        560      0.784      0.855      0.883      0.596



      Epoch    GPU_mem   box_loss   cls_loss   dfl_loss  Instances       Size


      29/50      10.8G     0.9507     0.6713      1.187        279        640:  71%|███████▏  | 10/14 [00:07<00:03,  1.31it/s]